In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix


import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

C:\Users\navee\anaconda3\envs\testing_clone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [3]:
from custom import custom_unet_generator_v2,custom_unet_descriminator_dilated_v2,custom_unet_descriminator_v2
generator_g = custom_unet_generator_v2()
generator_f = custom_unet_generator_v2()
print(generator_g.summary())

discriminator_x = custom_unet_descriminator_v2(target=False)
discriminator_y = custom_unet_descriminator_v2(target=False)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 36864       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 128 73728       conv2d_1[0][0]                   
______________________________________________________________________________________________

In [4]:
LAMBDA = 10
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [5]:
# checkpoint_path = "./checkpoints/train/"
checkpoint_path = "./checkpoints/best/"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=50)

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.checkpoints[8]) #best checkpoint
#   print ('Latest checkpoint restored!!')

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')
  print(ckpt_manager.latest_checkpoint)

./checkpoints/best/ckpt-9


In [6]:
# print(ckpt_manager.checkpoints[8])

In [7]:
def generate_images_eval(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()
  return prediction

In [8]:
import tensorflow as tf

dataset, metadata = tfds.load('cycle_gan/horse2zebra', with_info=True, as_supervised=True)
test_horses = dataset['testA']

test_horses_np = []

for image_tuple in test_horses:
    image, _ = image_tuple  # unpack the tuple into individual elements
    image_np = image.numpy()
    test_horses_np.append(image_np)

print(len(test_horses_np))  # output: number_of_samples
print(test_horses_np[0].shape)  # output: (height, width, channels)


120
(256, 256, 3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
ind=0
cyclic_loss = 0
for input_image in test_horses_np:
    
    input_image = normalize(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    reconstructed_image = generate_images_eval(generator_g, input_image)
    cycled_input_image = generator_f(reconstructed_image, training=False)
    cyclic_loss = cyclic_loss + calc_cycle_loss(input_image, cycled_input_image).numpy()
    #below code is to save real image,generated image and cycled image
#     save1 = np.array(reconstructed_image)
#     save1 = (save1 + 1) * 127.5
#     save1 = np.squeeze(save1, axis=0)
#     save1 = array_to_img(save1)
    
#     ori = np.array(input_image)
#     ori = (ori + 1) * 127.5
#     ori = np.squeeze(ori, axis=0)
#     ori = array_to_img(ori)
    

#     cycled_input_image = generator_f(reconstructed_image, training=False)
    
#     save2 = np.array(cycled_input_image)
#     save2 = (save2 + 1) * 127.5
#     save2 = np.squeeze(save2, axis=0)
#     save2 = array_to_img(save2)
    
#     stack = np.hstack((ori,save1))
#     stack = np.hstack((stack,save2))
#     stack = cv2.cvtColor(stack,cv2.COLOR_BGR2RGB)
#     name=str(ind)+".png"
#     cv2.imwrite("E:/computer vision/project/changes/submisssion/data/model3/horse_cycledhorse/"+name, np.array(stack))
#     ind=ind+1

print("cyclic_loss=",cyclic_loss/len(test_horses))

In [9]:
import tensorflow as tf

dataset, metadata = tfds.load('cycle_gan/horse2zebra', with_info=True, as_supervised=True)

test_zebras = dataset['testB'] #loads zebras

test_zebras_np = []

for image_tuple in test_zebras:
    image, _ = image_tuple  # unpack the tuple into individual elements
    image_np = image.numpy()
    test_zebras_np.append(image_np)

print(len(test_zebras))  # output: number_of_samples
print(test_horses_np[0].shape)  # output: (height, width, channels)


140
(256, 256, 3)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy
import cv2
from tensorflow.keras.preprocessing.image import array_to_img
cyclic_loss = 0
ind=0
for input_image in test_zebras_np:
    
    input_image = normalize(input_image)
    input_image = np.expand_dims(input_image, axis=0)
    reconstructed_image = generate_images_eval(generator_f, input_image)
    cycled_input_image = generator_g(reconstructed_image, training=False)
    cyclic_loss = cyclic_loss + calc_cycle_loss(input_image, cycled_input_image).numpy()
#     below codes saves original image, generated image and cycled image
#     save1 = np.array(reconstructed_image)
#     save1 = (save1 + 1) * 127.5
#     save1 = np.squeeze(save1, axis=0)
#     save1 = array_to_img(save1)
    
#     ori = np.array(input_image)
#     ori = (ori + 1) * 127.5
#     ori = np.squeeze(ori, axis=0)
#     ori = array_to_img(ori)
    
    
#     save2 = np.array(cycled_input_image)
#     save2 = (save2 + 1) * 127.5
#     save2 = np.squeeze(save2, axis=0)
#     save2 = array_to_img(save2)
    
#     stack = np.hstack((ori,save1))
#     stack = np.hstack((stack,save2))
#     stack = cv2.cvtColor(stack,cv2.COLOR_BGR2RGB)
#     name=str(ind)+".png"
#     cv2.imwrite("E:/computer vision/project/changes/submisssion/data/model3/zebra_cycledzebra/"+name, np.array(stack))
#     ind=ind+1

print("cyclic_loss=",cyclic_loss/len(test_horses))